In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# covide-19 글로벌 kaggle 데이터
kag = pd.read_csv('./data/corona-virus-report/covid_19_clean_complete.csv')
kag['Date'] = pd.to_datetime(kag['Date'])

In [4]:
kag.drop(['Province/State', 'Lat', 'Long'], axis=1, inplace=True)

In [9]:
kag.head()

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,0,0,0
1,Albania,2020-01-22,0,0,0
2,Algeria,2020-01-22,0,0,0
3,Andorra,2020-01-22,0,0,0
4,Angola,2020-01-22,0,0,0


In [11]:
pivoted_kag = kag.pivot_table(index = 'Date', columns = 'Country/Region', values='Confirmed', aggfunc='sum')

In [ ]:
주요 국가만 추출해서 사용한다
주요 국가는 질병관리본부 브리핑에 언급되는 나라들로 구성
유럽 국가
영국, 프랑스, 독일, 스페인, 이탈리아
아시아
중국, 대만, 태국, 베트남, 일본, 우즈베키스탄, 카자흐스탄, 파키스탄, 인도, 싱가포르, 아랍에미리트

In [76]:
pivoted_kag[['France']]

Country/Region,France
Date,
2020-01-22,0
2020-01-23,0
2020-01-24,2
2020-01-25,3
2020-01-26,3
2020-01-27,3
2020-01-28,4
2020-01-29,5
2020-01-30,5


In [36]:
euro = ['United Kingdom', 'France', 'Germany', 'Spain','Italy']
china = ['China']
asia = ['Japan', 'Kazakhstan', 'Taiwan*', 'Uzbekistan', 'Thailand', 'Vietnam', 'India', 'Pakistan', 'United Arab Emirates', 'Singapore']
america = ['US']

In [37]:
new_glob1 = pivoted_kag[euro+china+asia+america]

In [41]:
new_glob1 = pd.DataFrame()

In [42]:
new_glob1['EURO'] = pivoted_kag[euro].sum(axis=1)
new_glob1['CHINA'] = pivoted_kag[china].sum(axis=1)
new_glob1['ASIA'] = pivoted_kag[asia].sum(axis=1)
new_glob1['AMERICA'] = pivoted_kag[america].sum(axis=1)

In [44]:
new_glob1.head()

,EURO,CHINA,ASIA,AMERICA
Date,,,,
2020-01-22,0,548,5,1
2020-01-23,0,643,9,1
2020-01-24,2,920,15,2
2020-01-25,3,1406,17,2
2020-01-26,3,2075,22,5


In [32]:
for i in pivoted_kag.columns:
    print(i)

Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burma
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Comoros
Congo (Brazzaville)
Congo (Kinshasa)
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Cyprus
Czechia
Denmark
Diamond Princess
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Fiji
Finland
France
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Holy See
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
MS Zaandam
Madagascar
Malawi
Malaysia
Maldives
Mali
Malta
Mauritania
Mauriti

In [ ]:
# 각 컬럼별 일별 데이터 산출

cols = list(kag_dated_1.columns[1:])
for col in cols:
    col_name = "Day"+str(col)
    kag_dated_1['{}'.format(col_name)] = kag_dated_1['{}'.format(col)].diff()
    kag_dated_1['{}'.format(col_name)].iloc[0] = 0

# 1. 글로벌 확진자 데이터

In [2]:
# 글로벌 확진 데이터
glob = pd.read_csv('./data/global.csv', thousands = ',')

In [3]:
glob1 = glob[glob['국가']=='발생']

In [4]:
# 날짜 변수 변경
glob1['날짜'] = glob1['날짜'].astype(int)
glob1['날짜'] = glob1['날짜'].astype('object')
# 날짜 형태로 변경
glob1['날짜'] = pd.to_datetime(glob1['날짜'], format='%Y%m%d')
# na 값 0으로 처리
glob1.fillna(0, inplace=True)

In [5]:
glob1.head()

,날짜,국가,중국,홍콩,대만,마카오,일본,싱가포르,태국,말레이시아,...,그린랜드,맨섬,터크스 케이커스 제도,앵귈라,영국령 버진아일랜드,북마리아나제도,카리브 네덜란드,포클랜드제도,생피에르미클롱,합계
0,2020-01-20,발생,201.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,204.0
2,2020-01-21,발생,219.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,222.0
4,2020-01-22,발생,309.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,314.0
6,2020-01-23,발생,571.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,580.0
8,2020-01-24,발생,830.0,2.0,1.0,2.0,2.0,1.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,845.0


## 국가 확인

In [6]:
glob.columns

Index(['날짜', '국가', '중국', '홍콩', '대만', '마카오', '일본', '싱가포르', '태국', '말레이시아',
       ...
       '그린랜드', '맨섬', '터크스 케이커스 제도', '앵귈라', '영국령 버진아일랜드', '북마리아나제도', '카리브 네덜란드',
       '포클랜드제도', '생피에르미클롱', '합계'],
      dtype='object', length=219)

In [7]:
glob1.drop('국가', axis=1, inplace=True)

In [8]:
glob1.rename(columns={'날짜':'Date'}, inplace=True)

In [9]:
# 특수 문자 들어가 있는 컬럼 확인
for i in list(glob1.columns):
    if glob1['{}'.format(i)].dtypes == 'object':
        print(i)

독일
스페인


In [10]:
glob1['독일']=glob1['독일'].str.replace('*', "")
glob1['스페인']=glob1['스페인'].str.replace('*', "")
glob1['독일']=glob1['독일'].str.replace(',', "")
glob1['스페인']=glob1['스페인'].str.replace(',', "")

In [140]:
# 결측치 확인
glob1.isnull().sum().sum()

0

In [12]:
# 0으로 대체
glob1.fillna(0,inplace=True)

In [13]:
# object -> int
glob1['독일'] = glob1['독일'].astype('int64')
glob1['스페인'] = glob1['스페인'].astype('int64')

In [15]:
glob1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 202
Columns: 218 entries, Date to 합계
dtypes: datetime64[ns](1), float64(215), int64(2)
memory usage: 174.5 KB


In [16]:
# 전체 데이터 int형태로 변경
glob1.iloc[:,1:] = glob1.iloc[:,1:].astype('int64')

## 국가 추출
- 주요 국가만 추출해서 사용한다
- 주요 국가는 질병관리본부 브리핑에 언급되는 나라들로 구성

#### 유럽 국가
- 영국, 프랑스, 독일, 스페인, 이탈리아

#### 아시아
- 중국, 대만, 태국, 베트남, 일본, 우즈베키스탄, 카자흐스탄, 파키스탄, 인도, 싱가포르, 아랍에미리트

#### 미주
- 미국

In [17]:
glob1.set_index('Date', inplace=True)

In [37]:
euro = ['영국', '프랑스', '독일', '스페인', '이탈리아']
china = ['중국', '홍콩', '마카오']
asia = ['대만', '태국', '베트남', '일본', '우즈베키스탄', '카자흐스탄', '파키스탄', '인도', '싱가포르', '아랍에미리트']
america = ['미국']

In [41]:
glob2 = pd.DataFrame()

In [42]:
glob2['EURO'] = glob1[euro].sum(axis=1)
glob2['CHINA'] = glob1[china].sum(axis=1)
glob2['ASIA'] = glob1[asia].sum(axis=1)
glob2['AMERICA'] = glob1[america].sum(axis=1)

In [43]:
new_glob=glob2.reset_index()

In [45]:
new_glob.head()

,Date,EURO,CHINA,ASIA,AMERICA
0,2020-01-20,0,201,3,0
1,2020-01-21,0,219,3,0
2,2020-01-22,0,309,4,1
3,2020-01-23,0,573,6,1
4,2020-01-24,0,834,10,1


# 2. 로밍데이터

In [45]:
# 로밍데이터
roam1 = pd.read_csv('./data/corona_contest_data_0406/2. Roaming_data.csv')
roam2 = pd.read_csv('./data/corona_contest_data_0429/2. Roaming_data.csv')

In [46]:
roam = pd.concat([roam1, roam2])
roam.head()

,return,iso,arrival,departure,count
0,20200131,ad,20200130,20200130,3
1,20200131,ae,20191231,20200125,3
2,20200131,ae,20191231,20200131,3
3,20200131,ae,20200102,20200102,3
4,20200131,ae,20200104,20200131,3


In [47]:
print(roam1.shape)
print(roam2.shape)
print(roam.shape)

(212325, 5)
(5480, 5)
(217805, 5)


In [48]:
# 날짜 변수로 변경
roam['return'] = pd.to_datetime(roam['return'], format='%Y%m%d')
roam['arrival'] = pd.to_datetime(roam['arrival'], format='%Y%m%d')
roam['departure'] = pd.to_datetime(roam['departure'], format='%Y%m%d')

# 파생변수 생성
# arrival과 return의 차를 내서 해외에 체류한 일자수를 계산함
roam['stay'] = roam['return'] - roam['arrival']
roam.rename(columns={'return':'Date'}, inplace=True)

In [49]:
euro_dict = {'gb':'영국', 'fr':'프랑스', 'de':'독일', 'es':'스페인', 'it':'이탈리아'}

In [50]:
china_dict = {'cn':'중국', 'hk':'홍콩', 'mo':'마카오'}
asia_dict = {'tw':'대만', 'th':'태국', 'vn':'베트남', 'jp':'일본', 'uz':'우즈베키스탄', 'kz':'카자흐스탄', 'pk':'파키스탄', 'in':'인도', 'sg':'싱가포르', 'ae':'아랍에미리트'}

In [51]:
america_dict = {'us':'미국'}

In [52]:
pivoted_roam = roam.pivot_table(index='Date', 
                 columns='iso',
                 values='count',
                 aggfunc='sum')

In [53]:
new_roam = pd.DataFrame()

In [54]:
new_roam['EURO'] = pivoted_roam[list(euro_dict.keys())].sum(axis=1)
new_roam['CHINA'] = pivoted_roam[list(china_dict.keys())].sum(axis=1)
new_roam['ASIA'] = pivoted_roam[list(asia_dict.keys())].sum(axis=1)
new_roam['AMERICA'] = pivoted_roam[list(america_dict.keys())].sum(axis=1)

In [55]:
new_roam.reset_index(inplace=True)

# Merge Datesets

In [57]:
df = pd.merge(new_glob1, new_roam, on='Date', suffixes=('_conf', '_roam'),)

In [58]:
df.tail()

,Date,EURO_conf,CHINA_conf,ASIA_conf,AMERICA_conf,EURO_roam,CHINA_roam,ASIA_roam,AMERICA_roam
93,2020-04-24,855575,83899,78546,905358,39.0,197.0,311.0,99.0
94,2020-04-25,868982,83909,82812,938154,34.0,73.0,298.0,108.0
95,2020-04-26,879336,83912,86839,965785,62.0,126.0,294.0,100.0
96,2020-04-27,891948,83918,91150,988197,50.0,67.0,194.0,106.0
97,2020-04-28,903593,83940,94606,1012582,15.0,143.0,224.0,102.0


## 한국 해외유입확진자 데이터 로드

In [66]:
ov = pd.read_csv('./data/oversea_inflow.csv')
# 불필요 row 삭제
ov.drop([0,106], axis=0, inplace=True)

In [67]:
ov.rename(columns={'date':'Date', 'oversea_inflow':'TARGET'}, inplace=True)

In [68]:
ov['Date'] = pd.to_datetime(ov['Date'])

In [69]:
new_ov = ov[['Date','TARGET']]
new_ov.head()

,Date,TARGET
1,2020-01-21,1
2,2020-01-22,1
3,2020-01-23,1
4,2020-01-24,2
5,2020-01-25,2


In [70]:
df = pd.merge(df, new_ov, on='Date')

In [71]:
# 데이터 저장
df.to_csv('./data/total.csv')